In [2]:
import re
import requests
import pandas as pd
import numpy as np
import json
from IPython.display import display
import sys
sys.setrecursionlimit(6000)

#### Set up Mongo DB

In [3]:
!pip install pymongo
import pymongo
cli = pymongo.MongoClient(host='34.210.97.79')

    100% |████████████████████████████████| 368kB 1.4MB/s ta 0:00:01
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
cli.database_names()

['admin', 'local', 'wiki_mongo_database', 'wiki_mongo_db']

In [5]:
#Create a reference to a database
wiki_mongo_database = cli.wiki_mongo_database
cli.database_names()

['admin', 'local', 'wiki_mongo_database', 'wiki_mongo_db']

In [6]:
#Create a reference to a collection
wiki_mongo_collection = cli.wiki_mongo_database.wiki_mongo_collection
cli.database_names()

['admin', 'local', 'wiki_mongo_database', 'wiki_mongo_db']

### Using Wikipedia Python Library

In [15]:
! pip install wikipedia
import wikipedia

  Running setup.py bdist_wheel for wikipedia ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/bf/87/25/df698dd7b66a42c1c5f3bd36f8155d4518d210f5e2c128b440
Successfully built wikipedia
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [55]:
DEBUG = True
VERBOSE = True

#if VERBOSE: print ('generating query string fpr pageid{}'.format(pageid))
#if DEBUG: print(response_json)
def qry_category(category):
    new_str = re.sub('\s','+', category) 
    r = requests.get('http://en.wikipedia.org/w/api.php?\
                     action=query&\
                     format=json&\
                     list=categorymembers&\
                     cmtitle=Category%3A+{}&\
                     cmlimit=max'.format(new_str))
    re.sub('\s','+', category)
    return pd.DataFrame(r.json()['query']['categorymembers'])


def get_page_df(category):
    ml_df = qry_category(category)
    pages_list=[]
    page_df = ml_df[~ml_df['title'].str.contains('Category:')]
    page_df['category'] = category
    pages_list.append(page_df)
    category_df = ml_df[ml_df['title'].str.contains('Category:')]
    categories = category_df[category_df['title'].\
                             str.contains('Category:')]['title'].str.replace('Category:',"")
    if (category_df.shape[0]>0):
    #if (category_df.shape[0]>0 and category_df.shape[0]<30):     
        try:
            for category in categories:
                pages_list.append(get_page_df(category))
                if VERBOSE: print ('current_category: {}'.format(category))
        except KeyError:
            pass
    pages_df = pd.concat(pages_list)
    pages_df = pages_df.sort_values(by='title', axis=0, ascending=True)
    return pages_df

def get_pages_for_category(category):
    df =get_page_df(category)
    df['category'] = category
    df = df.reset_index()
    return df

def insert_to_wiki_mongo(Category='main page'):
    df = get_page_df(Category)
    dict_list = []
    for i in df['pageid']:
        try:
            page= wikipedia.page(pageid=i)
            if VERBOSE: print(page.pageid,page.title)
            dict_ = {
                "pageid":page.pageid,
                "title":page.title,
                "content":page.content,
                "category":Category,
                   }
           
            wiki_mongo_collection.update_one(dict_,{'$set': dict_}, upsert=True)
        except AttributeError:
            pass
        except ValueError:
            print(dict_)
            raise
    return cli.database_names(), wiki_mongo_collection.count()

In [61]:
#insert_to_wiki_mongo('machine learning')
#insert_to_wiki_mongo('Microsoft Office')
#insert_to_wiki_mongo('Geographic information systems')
#insert_to_wiki_mongo('History of the petroleum industry')
#insert_to_wiki_mongo('Yoga techniques')
#insert_to_wiki_mongo('Business intelligence')
#insert_to_wiki_mongo('Culture_in_Taipei')
#insert_to_wiki_mongo('Statistical_methods')
#insert_to_wiki_mongo('Diets')
#insert_to_wiki_mongo('Social_media')
#insert_to_wiki_mongo('History_of_Taiwan')
#insert_to_wiki_mongo('Taiwan_under_Japanese_rule')
#insert_to_wiki_mongo('Taiwan independence movement')
#insert_to_wiki_mongo('Yogis')
#insert_to_wiki_mongo('Yoga')
#insert_to_wiki_mongo('Taiwan')

#### Too many sub categories for recursive
#### insert_to_wiki_mongo('Business software')


In [10]:
wiki_mongo_collection.count()

5665

In [11]:
wiki_df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_mongo_collection.find({})))
wiki_df.head()

,_id,category,clean_content,content,pageid,title
0,591b9c60ad28cdd96d814531,machine learning,approximate near neighbor search special case ...,(1+ε)-approximate nearest neighbor search is a...,32003319,(1+ε)-approximate nearest neighbor search
1,591b9c65ad28cdd96d814535,machine learning,adaline adaptive linear neuron later adaptive ...,ADALINE (Adaptive Linear Neuron or later Adapt...,8220913,ADALINE
2,591b9c66ad28cdd96d814538,machine learning,ai formally know dartmouth artificial intellig...,"AI@50, formally known as the ""Dartmouth Artifi...",27562454,AI@50
3,591b9c67ad28cdd96d81453a,machine learning,aiva artificial intelligence virtual artist de...,AIVA (Artificial Intelligence Virtual Artist) ...,52642349,AIVA
4,591b9c6aad28cdd96d81453e,machine learning,aixi ai si theoretical mathematical formalism ...,AIXI ['ai̯k͡siː] is a theoretical mathematical...,30511763,AIXI


In [14]:
wiki_df[['pageid']].\
    groupby(wiki_df['category']).count()

,pageid
category,
Business intelligence,1174
Geographic information systems,470
Microsoft Office,109
Petroleum,1414
Statistical_methods,345
Taiwan,582
Yoga,494
machine learning,1077


In [59]:
#wiki_mongo_collection.update_many({"category":'Yogis'}, {'$set': {"category":'Yoga'}})

In [ ]:
# wiki_mongo_collection.remove({"category":'Petroleum'})
# wiki_mongo_collection.drop()